In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import tqdm

# Read image
files  = sorted(glob.glob("/ssd_scratch/cvit/anirudhkaushik/dataset/ISIC_2019/ISIC_2019_Training_Input/*.jpg"))
print(len(files))
# for f in tqdm.tqdm(files):
#     img = cv2.imread(f, cv2.IMREAD_COLOR)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     # resize to 224x224
#     img = cv2.resize(img, (224, 224),cv2.INTER_AREA )
#     # save over original
#     cv2.imwrite(f, img)


In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision.models import ResNet101_Weights, resnet101

# Load pretrained model
# model = resnet101(weights=ResNet101_Weights.IMAGENET1K_V2)
model = resnet101()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd

df_train = pd.read_csv("/ssd_scratch/cvit/anirudhkaushik/dataset/ISIC_2019/ISIC_2019_Training_GroundTruth.csv")
print(df_train.head())

In [ ]:
from sklearn.model_selection import train_test_split
import os
def load_isic_2019():
    root = "/ssd_scratch/cvit/anirudhkaushik/dataset/ISIC_2019"
    """
    Load ISIC_2019 dataset and convert it to IIRC format

    Args:
        root (string): The location of the dataset
        intask_valid_train_ratio (float): the percentage of the training set to be taken for the in-task validation set
            , a training-like validation set used for valdation during the task training (default: 0.1)
        posttask_valid_train_ratio (float): the percentage of the training set to be taken for the post-task validation
            set, a test-like validation set used for valdation after the task training (default: 0.1)

    Returns:
        Dict[str, DatasetStructType]: datasets, a dictionary with the keys corresponding to the four splits (train,
        intask_validation, posttask_validation, test), and the values being a list of the samples that belong to
        each split (with the images provided in Image.Image type) in the DatasetTypeStruct structure
    """
    raw_data_meta_df = pd.read_csv(root+'/ISIC_2019_Training_GroundTruth.csv')

    isic_data_map = {
        "MEL": "Melanoma",  
        "NV": "Melanocytic_nevus" ,
        "BCC": "Basal_cell_carcinoma",
        "AK": "Actinic_keratosis",
        "BKL": "Benign_keratosis",
        "DF": "Dermatofibroma",
        "VASC": "Vascular_lesion",
        "SCC": "Squamous_cell_carcinoma"
    }
    
    labels = list(raw_data_meta_df.columns[1:-1])
    class_to_idx = {isic_data_map[label]: idx for idx, label in enumerate(labels)}

    num_samples_nevus = 0
    num_samples_mel = 0

    X = raw_data_meta_df.iloc[:]['image'] # only image names, not actual images
    y = raw_data_meta_df.iloc[:, 1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify=y)

    raw_data_train = []
    for ind  in range(len(X_train)):
        img_name = X_train.iloc[ind]
        labels = y_train.iloc[ind]
        label = labels[labels == 1].index[0]
        if label != "MEL" and label != "NV" : # only take MEL and NV
            continue
        # if label == "NV":
        #     if num_samples_nevus >= 4070:
        #         continue
        #     num_samples_nevus += 1
        # if label == "MEL":
        #     num_samples_mel += 1
        image = cv2.imread(os.path.join(root, "ISIC_2019_Training_Input", img_name+".jpg"), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (224, 224), cv2.INTER_AREA) # remove later
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        label = class_to_idx[isic_data_map[label]] 
        raw_data_train.append((image, label))

    print(f"Train set, Number of nevus samples: {num_samples_nevus}\n Number of melanoma samples: {num_samples_mel}")
    num_samples_mel = 0
    num_samples_nevus = 0

    raw_data_test = []
    for ind  in range(len(X_test)):
        img_name = X_test.iloc[ind]
        labels = y_test.iloc[ind]
        label = labels[labels == 1].index[0]
        if label != "MEL" and label != "NV" : # only take MEL and NV
            continue
        # if label == "NV":
        #     if num_samples_nevus >= 452:
        #         continue
        #     num_samples_nevus += 1
        # if label == "MEL":
        #     num_samples_mel += 1
        image = cv2.imread(os.path.join(root, "ISIC_2019_Training_Input", img_name+".jpg"), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (224, 224), cv2.INTER_AREA) # remove later, inter area is for making it smaller, for making it larger use inter linear
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        label = class_to_idx[isic_data_map[label]]
        raw_data_test.append((image, label))

    print(f"Test set, Number of nevus samples: {num_samples_nevus}\n Number of melanoma samples: {num_samples_mel}")

    return raw_data_train, raw_data_test

In [ ]:
train, test = load_isic_2019()
print()

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# dataloader
class ISICDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img, label = self.data[idx]
        if self.transform:
            img = self.transform(img)
        return img, label
    
# transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225] )
])

# datasets
train_dataset = ISICDataset(train, transform=transform)
test_dataset = ISICDataset(test, transform=transform)

# dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
model.fc = nn.Linear(2048, 2)
model.load_state_dict(torch.load("./saved_models/melanocytic_exp2.pth"))
model = torch.nn.DataParallel(model)
model = model.to(device)

train_dataset_refined = []
test_dataset_refined = []

with torch.no_grad():
    model.eval()
    correct = 0
    total = 0
    for images, labels in train_dataset:
        images = images.unsqueeze(0)
        images = images.to(device)
        labels = torch.Tensor([labels]).long()
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        if (labels == 1 and predicted == 0) or (labels == 0):
            train_dataset_refined.append((images, labels.item()))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Train Accuracy: {100 * correct / total}")
    print(f"Size of refined train dataset: {len(train_dataset_refined)}")

    correct = 0
    total = 0
    for images, labels in test_dataset:
        images = images.unsqueeze(0)
        images = images.to(device)
        labels = torch.Tensor([labels]).long()
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        if (labels == 1 and predicted == 0) or (labels == 0):
            test_dataset_refined.append((images, labels.item()))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total}")
    print(f"Size of refined test dataset: {len(test_dataset_refined)}")



In [ ]:
train_dataset_refined_fixed = []
for images, labels in train_dataset_refined:
    train_dataset_refined_fixed.append((images.squeeze(0), labels))

test_dataset_refined_fixed = []
for images, labels in test_dataset_refined:
    test_dataset_refined_fixed.append((images.squeeze(0), labels))



In [ ]:
train_dataloader = DataLoader(train_dataset_refined_fixed, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset_refined_fixed, batch_size=32, shuffle=True)

In [ ]:
model = resnet101()

model.fc = nn.Linear(2048, 2)
model = torch.nn.DataParallel(model)
model = model.to(device)

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def test_model(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)

            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    print('\nTest set: Accuracy: {:.0f}%\n'.format(
            100. * correct / len(test_loader.dataset)))
    
    return correct / len(test_loader.dataset)

def train_model(model, train_loader, test_loader, optimizer, epochs):
    acc_list = []
    loss_list = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/100))
                running_loss = 0.0
        
        acc = test_model(model, test_loader)
        acc_list.append(acc)
        loss_list.append(running_loss)

    return acc_list, loss_list



In [ ]:
acc_list, loss_list = train_model(model, train_dataloader, test_dataloader, optimizer, 14)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(acc_list)
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.subplot(2, 1, 2)
plt.plot(loss_list)
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")

plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(acc_list)
plt.title("Accuracy")
plt.hlines(0.5, 0, 14, colors='r', linestyles='dashed', label='Baseline')
plt.legend()
plt.xlim(0, 15)
plt.ylim(0, 1)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.subplot(2, 1, 2)
plt.plot(loss_list)
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.tight_layout()
plt.show()

In [ ]:
final_test_acc = test_model(model, test_dataloader)
print("Final test accuracy: {:.0f}%".format(100. * final_test_acc))

In [ ]:
def classwise_acc(model, test_loader):
    classes = ("Melanoma", "Melanocytic Nevus")
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
    return [correct_pred["Melanoma"] / total_pred["Melanoma"], correct_pred["Melanocytic Nevus"] / total_pred["Melanocytic Nevus"]]


In [ ]:
classwise_score = classwise_acc(model, test_dataloader)
print("Melanoma class accuracy: {:.0f}%".format(100. * classwise_score[0]))
print("Melanocytic Nevus class accuracy: {:.0f}%".format(100. * classwise_score[1]))

In [ ]:
# mean and standard deviation of accuracy 
mean = np.mean(acc_list)
std = np.std(acc_list)
print("Mean accuracy: {:.2f}%".format(100. * mean))
print("Standard deviation of accuracy: {:.2f}%".format(100. * std))

In [ ]:
torch.save(model.module.state_dict(), "./saved_models/dysplastic_nevus_exp1.pth")